In [100]:
#!flask/bin/python
from flask import Flask, jsonify, abort, make_response, request,render_template
import pickle
import json
import os

In [101]:
f = open('classifier.pickle', 'rb')
classifier = pickle.load(f)
f.close()

In [102]:
f = open('vetoriza.pickle', 'rb')
vetoriza = pickle.load(f)
f.close()

In [103]:
def predicao(texto):
    vetor = []
    vetor.append(texto)
    vetorizacao = vetoriza.transform(vetor)
    resultado = classifier.predict(vetorizacao)[0]
    return resultado

In [132]:
app = Flask(__name__)

In [133]:
@app.errorhandler(404)
def not_found(error):
    return make_response(jsonify({'error': 'Not found'}), 404)

In [134]:
@app.errorhandler(400)
def not_found(error):
    return make_response(jsonify({'error': 'bad request'}), 400)

In [135]:
@app.errorhandler(500)
def not_found(error):
    return make_response(jsonify({'error': 'internal server erro'}), 500)

In [136]:
@app.route('/alethic/api/v1.0/ia', methods=['POST'])
def get_result():
    if not request.json or not 'texto' in request.json:
        abort(400)
    texto = request.json['texto']
    frases = texto.split('. ')
    vetor_resp = []
    for frase in frases:
        vetor_resp.append({'frase':frase,'resultado':predicao(texto)})
    respostas = {
        'respostas':vetor_resp
    }
    json_string = json.dumps(vetor_resp,ensure_ascii = False)
    response = make_response(json_string)
    response.headers['Content-Type'] = 'application/json; charset=utf-8'     
    return response

In [137]:
@app.route('/')
def render_static():
    return render_template('index.html')

In [ ]:
if __name__ == '__main__':
    port = int(os.environ.get("PORT", 5000))
    app.run(host='0.0.0.0',port=port)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [05/Dec/2019 19:26:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 19:26:16] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 19:38:28] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 19:38:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 19:38:43] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 19:39:09] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 19:39:12] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 20:01:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 20:01:27] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 20:16:33] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 20:17:32] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 20:17:40] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/2019 20:17:43] "POST /alethic/api/v1.0/ia HTTP/1.1" 200 -
127.0.0.1 - - [05/Dec/